# Power law fitting



## Initialise code

First, we need to load some packages and functions. This will enable us to reading data from csv files, make power law fits and to plot any results.

In [ ]:
from pyrootmemo.io import read_csv_roots
from pyrootmemo.fit import Powerlaw
import matplotlib.pyplot as plt

## Load data

There are three sources of data available in the "sbee2025/data" folder:

* tensile test data for Maize (Meijer et al (2024, see https://doi.org/10.1007/s11104-024-06507-y))
* tensile test data for *Lolium perenne* (Talveg dataset)
* tensile test data for *Onobrychis viciifolia* (Talveg dataset)

By default, this notebook loads the Maize data but you can easily change this out by changing the path to the relevant data file.

Alternatively, you can read in your own data by placing a comma seperated .csv file in the project folder and adjusting the path below. Your csv file must have (at least) two columns, one for diameter and one for tensile strength measurements. All column should be named in the .csv header row using the following convention:

* 'diameter_unit'
* 'tensile_strength_unit'

where 'unit' is the measurement unit in your data. For example, the default data had headers 'diameter_mm' and 'tensile_strength_MPa' for the columns containing diameter (measured in millimetres) and tensile strength values (measured in megapascals).

In [ ]:
# load tensile strength data from csv
data = read_csv_roots('data/tensile_maize_meijer2024.csv')
# data = read_csv_roots('data/tensile_lolium_perenne_talveg.csv')
# data = read_csv_roots('data/tensile_onobrychis_viciifolia_talveg.csv')


Your data is loaded into the object `data`. This object has attributes `diameter` and `tensile_strength` (and perhaps others too if you use your own data) that contain each lists of data. These data keeps its measurement units with it, so there is never any confusion as to what it units are! 

Let's plot a histogram showing the distribution of tested diameters.

In [ ]:
plt.hist(data.diameter.magnitude)
plt.xlabel('Diameter [' + str(data.diameter.units) + ']')
plt.ylabel('Number of roots [-]')

Let's also have a look at all diameter and tensile strength measurements in a scatter plot, so we know what we are trying to fit.

In [ ]:
plt.scatter(data.diameter.magnitude, data.tensile_strength.magnitude)
plt.xlabel('Diameter [' + str(data.diameter.units) + ']')
plt.ylabel('Tensile strength [' + str(data.tensile_strength.units) + ']')

## Fit

Let's now generate some power law fits for your data.

There are different fitting models to choose from in `pyrootmemo`. These models are further described in Meijer (2025), see https://doi.org/10.1007/s11104-024-07007-9. Currently, you can choose from:

| pyrootmemo name | distribution | scaling intra-diameter variation | notes
| --- | --- | --- | --- 
| `gamma` | gamma | scales with mean | |
| `gumbel` | gumbel | scales with mean | |
| `logistic` | logistic | scales with mean | |
| `lognormal` | lognormal | scales with mean | |
| `lognormal_uncorrected` | lognormal | scales with mean | not corrected for geometric mean |
| `normal` | normal | constant with diameter (in terms of strength) | same as least-squared regression of diameter--strength data |
| `normal_force` | normal | constant with diameter (in terms of force) | same as to least-squared regression of diameter--force at failure data |
| `normal_scaled` | normal | scales with mean | |
| `normal_freesd` | normal | varies with diameter according to seperate power law | |
| `uniform` | uniform | scales with mean | |
| `weibull` | weibull | scales with mean | |

We suggest you start with `normal`. This is the (conventional) least-squared regression method. Then try some of the other fits to see how the fit changes. 

In [ ]:
# define fit type
fit_model = 'normal'

# generate tensile strength fit object
fit = Powerlaw(
    data.diameter, 
    data.tensile_strength, 
    model = fit_model
    )

# show best-fitting power law parameters
print('Multiplier:', fit.multiplier)
print('Exponent:', fit.exponent)
# show Kolmogorov-Smirnov distance (smaller values indicate chosen distribution is more suitable)
print('Kolmogorov-Smirnov distance:', fit.ks_distance())
# show loglikelihood result (larger values are a measure for better fits)
print('Loglikelihood:', fit.loglikelihood())

# plot results (plus confidence and/or prediction intervals)
fit.plot(
    confidence = True, 
    prediction = True, 
    confidence_level = 0.95,
    prediction_level = 0.95
    )

The confidence interval shows you the band in which the **fit** will be, with 95% certainty. Thinner bands are better.

The prediction interval shows the band in which any **new data** is predicted to be, based on the current fit, with 95% probability. This interval should match the measured data as closely as possible for any diameter.

## Explore!

* What fitting model gives the most reliable results for your data?
* Try with a different set of data, e.g.:
  * *Lolium perenne*
  * *Onobrychis viciifolia*
  
  Is the 'best' model different?